In [6]:
import numpy as np
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.mask import mask
import os
import matplotlib.pyplot as plt

In [52]:
#read the polygons
k45_mines = gpd.read_file("data/polygons/global_mining_polygons_v2.gpkg")

#set the column called 'geometry' as the geometry of the GeoPandas
k45_mines = k45_mines.set_geometry("geometry")
k45_mines = k45_mines.to_crs("EPSG:4326")

k45_mines.head()

,ISO3_CODE,COUNTRY_NAME,AREA,geometry
0,GBR,United Kingdom,1.782044,"POLYGON ((-1.67130 55.07660, -1.67040 55.07710..."
1,FJI,Fiji,0.023405,"POLYGON ((177.85480 -17.51730, 177.85490 -17.5..."
2,NZL,New Zealand,0.042758,"POLYGON ((175.87200 -37.34490, 175.87180 -37.3..."
3,FJI,Fiji,0.248552,"POLYGON ((177.84260 -17.51850, 177.84370 -17.5..."
4,FJI,Fiji,0.132724,"POLYGON ((177.84950 -17.49510, 177.84920 -17.4..."


In [84]:
type(k45_mines.iloc[1]['geometry'])


shapely.geometry.polygon.Polygon

### Get aggregated NTL Emissions
1. Get sorted list of files in 'data/viirs-tif/mosaic'
2. open and loop trough files and compute ntl-emissions sums for each intervall and each mine
3. append ntl-emissions for each intervall as cloumn to the polygon DataFrame

In [57]:
all_tif_mosaics = sorted(os.listdir('/Users/flurin/Downloads/mosaic/'))
all_tif_mosaics

#If list includes .DS_Store file move to directory in terminal and use <find . -name '.DS_Store' -type f -delete> to remove it

['2012-mosaic.tif',
 '2013-mosaic.tif',
 '2014-mosaic.tif',
 '2015-mosaic.tif',
 '2016-mosaic.tif',
 '2017-mosaic.tif',
 '2018-mosaic.tif',
 '2019-mosaic.tif',
 '2020-mosaic.tif',
 '2021-mosaic.tif']

In [ ]:
#i switched i with j !!!

for i in range(len(all_tif_mosaics)):
    file = all_tif_mosaics[i]
    #print('data/viirs-tif/mosaic/' + file)
    viirs_data = rasterio.open('/Users/flurin/Downloads/mosaic/' + file)

    agg_ntl = []

    for j in range(len(k45_mines)):
        active_polygon = k45_mines.iloc[j]

        crop_img, crop_transform = mask(viirs_data, shapes = [active_polygon['geometry']], crop=True)

        agg_ntl.append(np.sum(crop_img))

    k45_mines['agg_ntl_' + all_tif_mosaics[i][:4]] = agg_ntl

In [ ]:
#polygons.to_csv('ann_ntl_emiss_2012-2021')